# Staging Code

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)

In [2]:
user_input = "Hello, suggest a trip plan in Tokyo"

messages = [
        {
            "role": "system", 
            "content": "Only suggest 1 activity for the trip plane"
        },
        {
            "role": "user",
            "content": user_input
        }
    ]

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

In [ ]:
re_json = completion.choices[0].message.content
print(re_json)

# GMAIL API

In [1]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

### Refactored Code

In [2]:
local = True

def get_credentials(token_path="token.json", scopes=None):
    '''
    Gets credentials from user
    Args:
        token_path: (str) the token.json file unique to local users
        scopes: (str) sets scope for get_credentials
    '''
    if scopes is None:
        scopes = ["https://www.googleapis.com/auth/gmail.readonly"]
    creds = None
    if os.path.exists(token_path):
        creds = Credentials.from_authorized_user_file(token_path, scopes)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            client_config = {
                "installed": {
                    "client_id": os.environ.get("GOOGLE_CLIENT_ID_DESK"),
                    "client_secret": os.environ.get("GOOGLE_CLIENT_SECRET_DESK"),
                    "auth_uri": "https://accounts.google.com/o/oauth2/auth",
                    "token_uri": "https://oauth2.googleapis.com/token"
                }
            }
            flow = InstalledAppFlow.from_client_config(client_config, scopes)
            creds = flow.run_local_server(port=0)
        with open(token_path, "w") as token_file:
            token_file.write(creds.to_json())
    return creds

def build_gmail_service(creds):
    return build("gmail", "v1", credentials=creds)

creds = get_credentials()

service = build_gmail_service(creds)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=137875663308-6t6h1t063s9tql86gm95c4isqnplteud.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A56999%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=5S5gNppVW1ubIgeTFeScMBZmT7AmA3&access_type=offline


### Lists Past Messages

In [5]:
# Id's for messages
results = service.users().messages().list(userId="me").execute()["messages"]

print(results)

[{'id': '19602cdf64f5bfe7', 'threadId': '195e3e39dd480b99'}, {'id': '1960287bdc9c9340', 'threadId': '1960287798c6e21d'}, {'id': '196022898c6b9ac7', 'threadId': '196022898c6b9ac7'}, {'id': '1960220b7a16d1e3', 'threadId': '1960220b7a16d1e3'}, {'id': '19602202869444ab', 'threadId': '19601842c3966ff6'}, {'id': '196021d4afc03bc6', 'threadId': '19601842c3966ff6'}, {'id': '196021ac9afa501e', 'threadId': '196021ac9afa501e'}, {'id': '1960202a6cee7f92', 'threadId': '1960202a6cee7f92'}, {'id': '19601f87a43ad654', 'threadId': '19601dcac5a7e97b'}, {'id': '19601f07f497a540', 'threadId': '19601f07f497a540'}, {'id': '19601dcac5a7e97b', 'threadId': '19601dcac5a7e97b'}, {'id': '19601b85b83304de', 'threadId': '19601b80c1134157'}, {'id': '19601b80c1134157', 'threadId': '19601b80c1134157'}, {'id': '196019b168e2c8c5', 'threadId': '196019afa063a532'}, {'id': '196019afa063a532', 'threadId': '196019afa063a532'}, {'id': '19601860d583a863', 'threadId': '19601860d583a863'}, {'id': '19601846a3df07f2', 'threadId': 

### Gets messages based on found ID

In [11]:
# for x in results:
#     print(x.get('id'))

In [29]:
res_messages = []

res_messages.append(service.users().messages().get(userId="me", id="195d36cc119281ab", format="full").execute().get("payload"))
res_messages.append(service.users().messages().get(userId="me", id="195e3e39dd480b99", format="full").execute().get("payload"))
# found snippet
for res in res_messages:
    for head in res.get("headers"):
        print(head.get('value')) if head['name'] == "Subject" else None

Re: Cookie Recipe/Ingredients for Cookies & Cocoa Thursday
[csstu] Hiring two summer research assistants


### Narrowing down the Return Message

All the infromation I need for this project is in the 'payload'

In [27]:
res = res_messages[0].get("payload")
res1 = res_messages[1].get("payload")


# base, contains headers and the data in ["parts"]
print(res)
print(res1)

{'partId': '', 'mimeType': 'multipart/alternative', 'filename': '', 'headers': [{'name': 'Delivered-To', 'value': 'mickey.zhang@colby.edu'}, {'name': 'Received', 'value': 'by 2002:a05:6a10:d1d5:b0:5ec:e349:56a1 with SMTP id m21csp3453854pxv;        Wed, 26 Mar 2025 10:07:55 -0700 (PDT)'}, {'name': 'X-Google-Smtp-Source', 'value': 'AGHT+IGJUgXh3E+OcnCAHJ9ekYMPMq2qZEZ9XimXmC3lVPCoF8O0Mvb9l8j6abYMz2M9B2dKJR9P'}, {'name': 'X-Received', 'value': 'by 2002:a05:6512:3e2a:b0:549:91c6:970d with SMTP id 2adb3069b0e04-54b011ce9a8mr196727e87.5.1743008874766;        Wed, 26 Mar 2025 10:07:54 -0700 (PDT)'}, {'name': 'ARC-Seal', 'value': 'i=1; a=rsa-sha256; t=1743008874; cv=none;        d=google.com; s=arc-20240605;        b=NR9NOFQIpH8gwHSon/pibibGu4PioQI78aa83xGUptIKfA6ZYJd8xI4ZQC3l28SYgh         B6H/uXl3qfkavq0hzG/OGA9TWSVAT00aX/MU5mnhN3PxEBUP0J6FscTFtcCq22C+Ol71         jr8gsyx782jy5TKp81Y72J3yks28Hl6/sAYat4Hwra26OYW8HS+0FRdTt/LCUDUl1pHB         enHvLG5SZST8L/Zmz+tFpIEL52+Dal9N6FA8eFGZH2VU6Mou55FD

### Retrieved the Sbuject Line for Gmails

In [18]:
for head in res.get("headers"):
    print(head.get('value')) if head['name'] == "Subject" else None

[csstu] Fwd: Hiring two summer research assistants


In [19]:
res.get("parts")

[{'partId': '0',
  'mimeType': 'text/plain',
  'filename': '',
  'headers': [{'name': 'Content-Type', 'value': 'text/plain; charset="UTF-8"'},
   {'name': 'Content-Transfer-Encoding', 'value': 'quoted-printable'}],
  'body': {'size': 3245,
   'data': 'SnVzdCBhIHJlbWluZGVyIHRoYXQgaWYgeW91IGFyZSBwbGFubmluZyB0byBhcHBseSB0byB0aGVzZSBwb3NpdGlvbnMsDQpwbGVhc2Ugc2VuZCBtZSB5b3VyIGFwcGxpY2F0aW9uIGJ5IFN1bmRheSwgQXByaWwgNiAoc2VlIGJlbG93IGZvciBtb3JlDQpkZXRhaWxzKS4NCg0KQmVzdCwNClRhaGl5YQ0KDQotLS0tLS0tLS0tIEZvcndhcmRlZCBtZXNzYWdlIC0tLS0tLS0tLQ0KRnJvbTogVGFoaXlhIENob3dkaHVyeSA8dGNob3dkaHVAY29sYnkuZWR1Pg0KRGF0ZTogU2F0LCBNYXIgMjksIDIwMjUgYXQgNTo1MOKAr1BNDQpTdWJqZWN0OiBIaXJpbmcgdHdvIHN1bW1lciByZXNlYXJjaCBhc3Npc3RhbnRzDQpUbzogQ29tcHV0ZXIgU2NpZW5jZSBzdHVkZW50cyA8Y3NzdHVAY29sYnkuZWR1Pg0KDQoNCkFyZSB5b3UgaW50ZXJlc3RlZCBpbiB3b3JraW5nIG9uIEFJLWJhc2VkIHNvbHV0aW9ucyBmb3IgdGhlIGVudmlyb25tZW50Pw0KDQoNClByb2YuIFRhaGl5YSBDaG93ZGh1cnkgaXMgbG9va2luZyB0byBoaXJlIHR3byBzdW1tZXIgcmVzZWFyY2ggYXNzaXN0YW50cyB0bw0Kd29yayBvbiBt

In [30]:
print(res.get("parts")[0].get("body").get("data"))

QXJlIHlvdSBpbnRlcmVzdGVkIGluIHdvcmtpbmcgb24gQUktYmFzZWQgc29sdXRpb25zIGZvciB0aGUgZW52aXJvbm1lbnQ_DQoNCg0KUHJvZi4gVGFoaXlhIENob3dkaHVyeSBpcyBsb29raW5nIHRvIGhpcmUgdHdvIHN1bW1lciByZXNlYXJjaCBhc3Npc3RhbnRzIHRvDQp3b3JrIG9uIG1hY2hpbmUgbGVhcm5pbmcgYW5kIGRhdGEgc2NpZW5jZS1yZWxhdGVkIHByb2plY3RzIGluIGhlciByZXNlYXJjaA0KZ3JvdXAsICpIdSptYW4sICpNYSpjaGkqTiplLCBhbmQgKkUqbnZpcm9ubWVudCAoSFVNQU5FKSBMYWIuIFN1Y2Nlc3NmdWwNCmNhbmRpZGF0ZXMgd2lsbCB3b3JrIGNsb3NlbHkgb24gdGhlIGZvbGxvd2luZyB0d28gcHJvamVjdHMgcmVsYXRlZCB0byAqQUkgZm9yDQplbnZpcm9ubWVudGFsIHN1c3RhaW5hYmlsaXR5LioNCg0KDQoqRGF0YS1kcml2ZW4gRm9yZWNhc3RpbmcgZm9yIFdhdGVyIFF1YWxpdHkqOiBJbiB0aGlzIHByb2plY3QsIHdlIHN0dWR5IHRoZQ0KZWZmZWN0cyBvZiBjbGltYXRlIGNoYW5nZSBvbiBhc3BlY3RzIG9mIG91ciBuYXR1cmFsIGVudmlyb25tZW50LiBUaGUNCnN1Y2Nlc3NmdWwgY2FuZGlkYXRlIHdpbGwgYXBwbHkgdmFyaW91cyBkYXRhLWRyaXZlbiBtb2RlbGluZyB0ZWNobmlxdWVzIHRvDQpwcmVkaWN0IGFsZ2FlIGJsb29tIGluIE1haW5lIGxha2VzIHVzaW5nIGEgZGF0YWJhc2UgY29sbGVjdGVkIGZyb20gb3ZlciA3MDANCmxha2VzIGluIE1haW5lLiBDb21wdXRlciB2aXNpb24g

In [ ]:
import base64

data_str = "QXJlIHlvdSBpbnRlcmVzdGVkIGluIHdvcmtpbmcgb24gQUktYmFzZWQgc29sdXRpb25zIGZvciB0aGUgZW52aXJvbm1lbnQ_DQoNCg0KUHJvZi4gVGFoaXlhIENob3dkaHVyeSBpcyBsb29raW5nIHRvIGhpcmUgdHdvIHN1bW1lciByZXNlYXJjaCBhc3Npc3RhbnRzIHRvDQp3b3JrIG9uIG1hY2hpbmUgbGVhcm5pbmcgYW5kIGRhdGEgc2NpZW5jZS1yZWxhdGVkIHByb2plY3RzIGluIGhlciByZXNlYXJjaA0KZ3JvdXAsICpIdSptYW4sICpNYSpjaGkqTiplLCBhbmQgKkUqbnZpcm9ubWVudCAoSFVNQU5FKSBMYWIuIFN1Y2Nlc3NmdWwNCmNhbmRpZGF0ZXMgd2lsbCB3b3JrIGNsb3NlbHkgb24gdGhlIGZvbGxvd2luZyB0d28gcHJvamVjdHMgcmVsYXRlZCB0byAqQUkgZm9yDQplbnZpcm9ubWVudGFsIHN1c3RhaW5hYmlsaXR5LioNCg0KDQoqRGF0YS1kcml2ZW4gRm9yZWNhc3RpbmcgZm9yIFdhdGVyIFF1YWxpdHkqOiBJbiB0aGlzIHByb2plY3QsIHdlIHN0dWR5IHRoZQ0KZWZmZWN0cyBvZiBjbGltYXRlIGNoYW5nZSBvbiBhc3BlY3RzIG9mIG91ciBuYXR1cmFsIGVudmlyb25tZW50LiBUaGUNCnN1Y2Nlc3NmdWwgY2FuZGlkYXRlIHdpbGwgYXBwbHkgdmFyaW91cyBkYXRhLWRyaXZlbiBtb2RlbGluZyB0ZWNobmlxdWVzIHRvDQpwcmVkaWN0IGFsZ2FlIGJsb29tIGluIE1haW5lIGxha2VzIHVzaW5nIGEgZGF0YWJhc2UgY29sbGVjdGVkIGZyb20gb3ZlciA3MDANCmxha2VzIGluIE1haW5lLiBDb21wdXRlciB2aXNpb24gYW5kIHRpbWUgc2VyaWVzIGFuYWx5c2lzIHRlY2huaXF1ZXMgd2lsbCBiZQ0KdXNlZCB0byBkZXNpZ24gaW50ZXJhY3RpdmUgdG9vbHMgYW5kIHN1cHBvcnQgcG9saWN5IHJlY29tbWVuZGF0aW9ucy4gVGhpcyBpcw0KYSBwcm9qZWN0IGluIHdoaWNoIHRoZSB0ZWFtIHdpbGwgd29yayBpbiBjb2xsYWJvcmF0aW9uIHdpdGggdGhlIENvbGJ5DQpDaGVtaXN0cnkgZGVwYXJ0bWVudCBhbmQgNyBMYWtlcyBBbGxpYW5jZS4NCg0KDQoqTGFyZ2UgbGFuZ3VhZ2UgbW9kZWwtYmFzZWQgYXBwbGljYXRpb24gZm9yIGNvbnNlcnZhdGlvbiBhd2FyZW5lc3M6ICpUaGlzDQpwcm9qZWN0IGlzIGEgY29sbGFib3JhdGl2ZSBwcm9qZWN0IHdpdGggdGhlIE1hcmluZSBEZWJyaXMgVHJhY2tlciB0ZWFtIGF0DQp0aGUgVW5pdmVyc2l0eSBvZiBHZW9yZ2lhLiBUaGUgc3VjY2Vzc2Z1bCBjYW5kaWRhdGUgd2lsbCB3b3JrIG9uIGRldmVsb3BpbmcNCmEgd2ViIGFwcGxpY2F0aW9uIHVzaW5nIGEgbGFyZ2UgZGF0YWJhc2UgYmFzZWQgb24gbWFyaW5lIGRlYnJpcyBwb2xpY3kNCnJlcG9ydHMgYW5kIGxhcmdlIGxhbmd1YWdlIG1vZGVscyB0byBhbGxvdyBzdGFrZWhvbGRlcnMgdG8gZW5nYWdlIHdpdGgNCmV4aXN0aW5nIGNvbnNlcnZhdGlvbiBlZmZvcnRzIGFuZCBkZXZlbG9wIGluZGl2aWR1YWwgYW5kIGNvbGxlY3RpdmUgYWN0aW9uDQpwbGFucy4NCg0KDQpTdWNjZXNzZnVsIGFwcGxpY2FudHMgc2hvdWxkIGJlOg0KDQogICAxLiBTa2lsbGVkIGluIFB5dGhvbiBwcm9ncmFtbWluZy4NCiAgIDIuIEhhdmUgdGFrZW4gQ1MgMjUxL0NTMjUyL0NTMzQzL0pQMjk3RC4gUmlzaW5nIGp1bmlvciBvciBzZW5pb3INCiAgIHByZWZlcnJlZC4NCiAgIDMuIEZhbWlsaWFyaXR5IHdpdGggbWFjaGluZSBsZWFybmluZy9kYXRhIHNjaWVuY2UgbGlicmFyaWVzIChzY2lraXQtbGVhcm4sDQogICBwYW5kYXMpIHByZWZlcnJlZC4NCiAgIDQuIENhbmRpZGF0ZXMgd2l0aCBhbiBpbnRlcmVzdCBpbiBkb2luZyBhbiBIb25vcnMgdGhlc2lzIGxhdGVyIGFyZQ0KICAgcHJlZmVycmVkLg0KICAgNS4gRmFtaWxpYXJpdHkgb3IgaW50ZXJlc3QgaW4gd2ViIGRldmVsb3BtZW50IHdpbGwgYmUgYSBwbHVzLg0KDQoNCkFwcGxpY2FudHMgc2hvdWxkIHNlbmQgKmEgQ1YgYW5kIGEgY292ZXIgbGV0dGVyIHRoYXQgaW5jbHVkZXMgMSkgV2h5IHRoZXkNCmFyZSBpbnRlcmVzdGVkIGluIHdvcmtpbmcgd2l0aCBtZSBvbiB0aGVzZSBwcm9qZWN0cyBhbmQgMikgYSoqIGxpc3Qgb2YgQ1MNCm9yIHJlbGV2YW50IGNvdXJzZXMgdGhleSB3aWxsIGhhdmUgdGFrZW4gYmVmb3JlIHN1bW1lciAyMDI1LioNCg0KDQpBcHBsaWNhdGlvbnMgc2hvdWxkIGJlIHNlbnQgdG8gdGFoaXlhLmNob3dkaHVyeUBjb2xieS5lZHUgYnkgKkFwcmlsIDYsIDIwMjUqDQoqLiAqDQoNCkkgd2lsbCByZWFjaCBvdXQgdG8gc2VsZWN0ZWQgY2FuZGlkYXRlcyBhYm91dCB0aGUgcmVtYWluaW5nIGhpcmluZyBwcm9jZXNzLg0KDQoNCkJlc3QsDQoNClRhaGl5YQ0KDQoNCi0tIA0KVGFoaXlhIENob3dkaHVyeQ0KQXNzaXN0YW50IFByb2Zlc3NvciwgQ29tcHV0ZXIgU2NpZW5jZQ0KRmFjdWx0eSwgRGF2aXMgSW5zdGl0dXRlIGZvciBBSQ0KQ29sYnkgQ29sbGVnZQ0KT2ZmaWNlOiBEYXZpcyAxMTIsIFdhdGVydmlsbGUsIE1FIDA0OTAxDQpXYW50IHRvIG1lZXQ_IENhbGVuZGFyIDxodHRwczovL2NhbGVuZGx5LmNvbS90YWhpeWFjL2RhdmlzYWktb2ZmaWNlLWhvdXI"

# ChatGPTed
def decode_base64_data(data_str):
    missing_padding = len(data_str) % 4
    if missing_padding:
        data_str += "=" * (4 - missing_padding)
    decoded_bytes = base64.urlsafe_b64decode(data_str.encode("utf-8"))
    return decoded_bytes.decode("utf-8", errors="replace")


print(decode_base64_data(data_str))

Are you interested in working on AI-based solutions for the environment?


Prof. Tahiya Chowdhury is looking to hire two summer research assistants to
work on machine learning and data science-related projects in her research
group, *Hu*man, *Ma*chi*N*e, and *E*nvironment (HUMANE) Lab. Successful
candidates will work closely on the following two projects related to *AI for
environmental sustainability.*


*Data-driven Forecasting for Water Quality*: In this project, we study the
effects of climate change on aspects of our natural environment. The
successful candidate will apply various data-driven modeling techniques to
predict algae bloom in Maine lakes using a database collected from over 700
lakes in Maine. Computer vision and time series analysis techniques will be
used to design interactive tools and support policy recommendations. This is
a project in which the team will work in collaboration with the Colby
Chemistry department and 7 Lakes Alliance.


*Large language model-based 

In [10]:
from bs4 import BeautifulSoup

class EmailData:
    
    def __init__(self, subject, message, html_tag_threshold=100, img_tag_threshold=10):
        """
        Initialize an EmailData instance. Optionally, filter out messages
        that contain too many HTML or image tags.
        """
        self.subject = subject
        # Filter the message if it appears to be "massive" in HTML content.
        if self._is_massive_html(message, html_tag_threshold, img_tag_threshold):
            self.message = "[Filtered: Message contains too many HTML elements or images]"
        else:
            self.message = message
        
    def _is_massive_html(self, message, html_threshold, img_threshold):
        """
        Check if a message has too many HTML tags or <img> tags.
        Returns True if the message exceeds the thresholds.
        """
        # Parse the message as HTML. If it's plain text, BeautifulSoup will have few tags.
        soup = BeautifulSoup(message, "html.parser")
        
        # Count all HTML tags
        html_tags = soup.find_all()
        # Count <img> tags specifically
        img_tags = soup.find_all('img')
        
        # Debug prints (optional)
        # print("Total HTML tags:", len(html_tags))
        # print("Image tags:", len(img_tags))
        
        # If the number of tags exceeds the thresholds, we consider it "massive"
        if len(html_tags) > html_threshold or len(img_tags) > img_threshold:
            return True
        return False
        
    def get_subject(self):
        '''
        gettr for subject
        '''
        return self.subject
    
    def get_subject(self):
        '''
        gettr for mnessage
        '''
        return self.subject
    
    def to_string(self):
        '''
        to_string method for EmailData
        '''
        return f">>>>>>>START OF '{self.subject}' Email <<<<<<<\nSUBJECT: {self.subject} \nMESSAGE: {self.message}\n >>>>>>>END OF '{self.subject}' Email <<<<<<<"

In [13]:
results = service.users().messages().list(userId="me").execute()["messages"]
# creates a list of messages found with id
res_messages = []
counter = 0
for message in results:
    if counter == 10:
        break
    id = message.get('id')
    extracted_data = service.users().messages().get(userId="me", id=id, format="full").execute()
    payload = extracted_data.get("payload")
    # Get subject line
    subject = None
    for head in payload.get("headers"):
        subject = head.get('value') if head['name'] == "Subject" else None
        if subject:
            break
    uncoded_message = payload.get("parts")[0]['body']['data'] if payload.get("parts")[0]['body']['size'] != 0 else ""
    message = decode_base64_data(uncoded_message)
    email_data = EmailData(subject, message)
    res_messages.append(email_data)
    counter += 1

In [14]:
for mail in res_messages:
    print(mail.to_string())

>>>>>>>START OF 'Fwd: Spreadsheet shared with you: "INT Program Spring Break Portland Trip (Responses)"' Email <<<<<<<
SUBJECT: Fwd: Spreadsheet shared with you: "INT Program Spring Break Portland Trip (Responses)" 
MESSAGE: ---------- Forwarded message ---------
From: Michael Freese (via Google Sheets) <drive-shares-dm-noreply@google.com
>
Date: Fri, Mar 28, 2025, 08:42
Subject: Spreadsheet shared with you: "INT Program Spring Break Portland
Trip (Responses)"
To: <qzhang27@colby.edu>


Michael Freese shared a spreadsheet
[image: Header profile photo]
Michael Freese (mfreese@colby.edu) has invited you to *edit* the following
spreadsheet:
INT Program Spring Break Portland Trip (Responses)
<https://urldefense.com/v3/__https://docs.google.com/spreadsheets/d/1XEGYTIEGmwrt6vqud0UpAK9tZUJntC21BUep1Gb2sEg/edit?usp=sharing_eil&ts=67e69917__;!!P_zEGVH0kSMiWA!AC7rPTncgF-HzE_2VYLhoVofHhZN4fpWQ-nbmkhC7vjHqWtcQJ0mPmlIhHzyHkgD0L55kDmEgMOErl3AfYt_UsG1Fm8_auK0$>

Open
<https://urldefense.com/v3/__http

In [15]:
user_input = [email.to_string() for email in res_messages]

messages = [
        {
            "role": "system", 
            "content": "Take a list of strings as input, each string in the list is an instance of an "
                        "email with a SUBJECT field and MESSAGE field. Make sense of each email and provide a sumamry of each "
                        "if the SUBJECT or MESSAGE of the eamil string is nonsensical i.e. not understandable humaan language "
                        "remove it from output completely, and continue to the next email"
                        "Make the summaries short and digestable while mainting integrity"
                        "output a STRING of a list of jsons containing the SUBJECT and MESSAGE summaries"
                        "Remove any excessive styling or syntax of your return"
        },
        {
            "role": "user",
            "content": str(user_input)
        }
    ]

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

In [16]:
import json

re_json = completion.choices[0].message.content
print(json.loads(re_json))

[{'SUBJECT': 'Fwd: Spreadsheet shared with you: "INT Program Spring Break Portland Trip (Responses)"', 'MESSAGE': 'Michael Freese shared a spreadsheet regarding responses for the INT Program Spring Break trip to Portland.'}, {'SUBJECT': 'Re: Portland trip itinerary today and contact info for Mickey and Michael', 'MESSAGE': 'Google Maps location for pickup is confirmed for 5 PM at the Visitors Center on the edge of Old Port.'}, {'SUBJECT': 'Re: Portland trip itinerary today and contact info for Mickey and Michael', 'MESSAGE': 'Mai requests clarification on the pickup location for the trip.'}, {'SUBJECT': 'Re: Portland trip itinerary today and contact info for Mickey and Michael', 'MESSAGE': 'Michelle Phan and Thu Nguyen are already near the Maine port, so no need to wait for them.'}, {'SUBJECT': 'Re: Portland trip itinerary today and contact info for Mickey and Michael', 'MESSAGE': 'Mickey suggests the best exit from JCPENNEY for the trip pickup and confirms the time.'}]
